In [13]:
import time
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import make_scorer, f1_score, recall_score
#Zadanie2
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
#Zadanie3
from sklearn.datasets import load_diabetes
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, r2_score, make_scorer
#Zadanie4
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

print('Zadanie 1')
# 1. Wczytaj zbiór danych IRIS
data = load_iris()
X, y = data.data, data.target

# A. Trening z użyciem cross_validate dla 5 podzbiorów
knn = KNeighborsClassifier(n_neighbors=3)
scorers = {
    "f1_macro": make_scorer(f1_score, average="macro"),
    "recall_macro": make_scorer(recall_score, average="macro"),
}

start_time = time.time()
results_5cv = cross_validate(
    knn,
    X,
    y,
    cv=5,
    n_jobs=4,
    verbose=2,
    scoring=scorers,
    return_train_score=True,
)
training_time = time.time() - start_time

# B. Wyniki dla każdego podzbioru
print("\nWyniki dla każdego podzbioru (5-cv):")
for i, f1_score in enumerate(results_5cv["test_f1_macro"]):
    print(f"Podzbiór {i + 1}: F1 = {f1_score:.4f}")

# C. Przeciętny wynik F1
mean_f1_5cv = results_5cv["test_f1_macro"].mean()
print(f"\nPrzeciętny F1 (5-cv): {mean_f1_5cv:.4f}")

# E. Wyniki czułości i jej średnia
print("\nWyniki czułości dla każdego podzbioru (5-cv):")
for i, recall in enumerate(results_5cv["test_recall_macro"]):
    print(f"Podzbiór {i + 1}: Recall = {recall:.4f}")
mean_recall_5cv = results_5cv["test_recall_macro"].mean()
print(f"\nPrzeciętna czułość (5-cv): {mean_recall_5cv:.4f}")

# F. Walidacja krzyżowa z 10 podzbiorami
results_10cv = cross_validate(
    knn,
    X,
    y,
    cv=10,
    n_jobs=4,
    verbose=2,
    scoring=scorers,
    return_train_score=True,
)

mean_f1_10cv = results_10cv["test_f1_macro"].mean()
print(f"\nPrzeciętny F1 (10-cv): {mean_f1_10cv:.4f}")

# G. Czas treningu
print(f"\nCzas treningu (5-cv): {training_time:.4f} sekundy")
print('')

print('Zadanie 2')
#Zadanie 2
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score, precision_score
import time

# Dane IRIS
X, y = load_iris(return_X_y=True)

# Zakresy parametrów do GridSearch
param_grid = {
    "max_depth": [1, 2, 4, 8, 16, 25, 64],
    "n_estimators": list(range(1, 26)),  # Od 1 do 25
}

# A. Przygotowanie modelu i GridSearch
rf = RandomForestClassifier(random_state=42)
scorers = {
    "f1_macro": make_scorer(f1_score, average="macro"),
    "precision_macro": make_scorer(precision_score, average="macro"),
}

start_time = time.time()
grid_search = GridSearchCV(
    rf,
    param_grid,
    cv=5,
    scoring=scorers,
    refit="f1_macro",  # Optymalizujemy pod kątem F1 score
    verbose=2,
    n_jobs=-1,
)
grid_search.fit(X, y)
training_time = time.time() - start_time

# B. Przeciętny wynik dla F1 score
mean_f1_score = grid_search.best_score_
print(f"Przeciętny F1 score (5-cv): {mean_f1_score:.4f}")

# C. Wynik F1 score dla każdego podzbioru
f1_scores = grid_search.cv_results_["mean_test_f1_macro"]
print("\nWyniki F1 score dla każdego podzbioru (5-cv):")
for i, f1 in enumerate(f1_scores):
    print(f"Podzbiór {i + 1}: F1 = {f1:.4f}")

# D. Wynik precyzji dla każdego podzbioru i jej średnia
precision_scores = grid_search.cv_results_["mean_test_precision_macro"]
print("\nWyniki precyzji dla każdego podzbioru (5-cv):")
for i, precision in enumerate(precision_scores):
    print(f"Podzbiór {i + 1}: Precision = {precision:.4f}")
mean_precision = precision_scores.mean()
print(f"Przeciętna precyzja (5-cv): {mean_precision:.4f}")

# E. Najlepsza konfiguracja
best_params = grid_search.best_params_
print(f"\nNajlepsza konfiguracja: {best_params}")

# F. Zmiana na walidację krzyżową z 10 podzbiorami
grid_search_10cv = GridSearchCV(
    rf,
    param_grid,
    cv=10,
    scoring=scorers,
    refit="f1_macro",
    verbose=2,
    n_jobs=-1,
)
grid_search_10cv.fit(X, y)
mean_f1_score_10cv = grid_search_10cv.best_score_
print(f"\nPrzeciętny F1 score (10-cv): {mean_f1_score_10cv:.4f}")

# G. Czas treningu
print(f"\nCzas treningu GridSearch (5-cv): {training_time:.4f} sekundy")

print('')
print('Zadanie 3')
#Zadanie3

# Załadowanie danych Diabetes
X, y = load_diabetes(return_X_y=True)

# Model: K Neighbors Regressor z 5 sąsiadami
knn_reg = KNeighborsRegressor(n_neighbors=5)

# A. Cross-Validation z 7 podzbiorami, 2 procesy, verbose=1
start_time = time.time()
cv_results = cross_validate(
    knn_reg,
    X,
    y,
    cv=7,
    n_jobs=2,
    verbose=1,
    scoring={"r2": make_scorer(r2_score), "mae": make_scorer(mean_absolute_error)},
    return_train_score=False,
)
training_time = time.time() - start_time

# B. Wyniki testu dla każdego podzbioru
print("\nWyniki testowe dla każdego podzbioru (R2):")
r2_scores = cv_results["test_r2"]
for i, score in enumerate(r2_scores, 1):
    print(f"Podzbiór {i}: R2 = {score:.4f}")

# C. Przeciętny wynik R2
mean_r2 = r2_scores.mean()
print(f"\nPrzeciętny wynik R2: {mean_r2:.4f}")

# D. Wyniki MAE dla każdego podzbioru
print("\nWyniki testowe dla każdego podzbioru (MAE):")
mae_scores = cv_results["test_mae"]
for i, score in enumerate(mae_scores, 1):
    print(f"Podzbiór {i}: MAE = {score:.4f}")

# E. Przeciętne MAE
mean_mae = mae_scores.mean()
print(f"\nPrzeciętne MAE: {mean_mae:.4f}")

# F. Walidacja krzyżowa z 10 podzbiorami
cv_results_10 = cross_validate(
    knn_reg,
    X,
    y,
    cv=10,
    n_jobs=2,
    verbose=1,
    scoring={"r2": make_scorer(r2_score), "mae": make_scorer(mean_absolute_error)},
    return_train_score=False,
)
mean_r2_10 = cv_results_10["test_r2"].mean()
print(f"\nPrzeciętny wynik R2 (10-cv): {mean_r2_10:.4f}")

# G. Czas treningu
print(f"\nCzas treningu (7-cv): {training_time:.4f} sekundy")
print('')

#Zadanie4
print('Zadanie 4')
# Model: Random Forest Regressor
rf_reg = RandomForestRegressor(random_state=42)

# Zakres parametrów do losowego przeszukiwania
param_distributions = {
    "max_depth": [1, 2, 4, 8, 9, 12, 81, 1024],
    "n_estimators": [3, 6],
}

# A. Randomized Search z 100 losowymi kombinacjami
random_search = RandomizedSearchCV(
    rf_reg,
    param_distributions,
    n_iter=100,
    scoring="r2",
    cv=5,
    verbose=1,
    n_jobs=-1,
    random_state=42,
)
start_time = time.time()
random_search.fit(X, y)
training_time = time.time() - start_time

# B. Najlepsza konfiguracja
best_params = random_search.best_params_
print(f"\nNajlepsza konfiguracja: {best_params}")

# C. Przeciętny wynik R2
mean_r2 = random_search.best_score_
print(f"\nPrzeciętny wynik R2 (5-cv): {mean_r2:.4f}")

# D. Wyniki R2 dla każdego podzbioru
cv_results = random_search.cv_results_
print("\nWyniki R2 dla każdego podzbioru:")
r2_scores = cv_results["mean_test_score"]
for i, score in enumerate(r2_scores, 1):
    print(f"Podzbiór {i}: R2 = {score:.4f}")

# E. Wyniki MAE dla każdego podzbioru i średnia
mae_scores = cross_val_score(
    random_search.best_estimator_,
    X,
    y,
    cv=5,
    scoring=make_scorer(mean_absolute_error),
)
print("\nWyniki MAE dla każdego podzbioru:")
for i, score in enumerate(mae_scores, 1):
    print(f"Podzbiór {i}: MAE = {score:.4f}")
mean_mae = mae_scores.mean()
print(f"Przeciętne MAE: {mean_mae:.4f}")

# F. Randomized Search z walidacją krzyżową (10 podzbiorów)
random_search_10cv = RandomizedSearchCV(
    rf_reg,
    param_distributions,
    n_iter=100,
    scoring="r2",
    cv=10,
    verbose=1,
    n_jobs=-1,
    random_state=42,
)
random_search_10cv.fit(X, y)
mean_r2_10cv = random_search_10cv.best_score_
print(f"\nPrzeciętny wynik R2 (10-cv): {mean_r2_10cv:.4f}")

# G. Czas treningu dla 200 kombinacji
random_search_200 = RandomizedSearchCV(
    rf_reg,
    param_distributions,
    n_iter=200,
    scoring="r2",
    cv=5,
    verbose=1,
    n_jobs=-1,
    random_state=42,
)
start_time_200 = time.time()
random_search_200.fit(X, y)
training_time_200 = time.time() - start_time_200
print(f"\nCzas treningu (200 kombinacji, 5-cv): {training_time_200:.4f} sekundy")


Zadanie 1


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 out of   5 | elapsed:    1.3s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished



Wyniki dla każdego podzbioru (5-cv):
Podzbiór 1: F1 = 0.9666
Podzbiór 2: F1 = 0.9666
Podzbiór 3: F1 = 0.9327
Podzbiór 4: F1 = 0.9666
Podzbiór 5: F1 = 1.0000

Przeciętny F1 (5-cv): 0.9665

Wyniki czułości dla każdego podzbioru (5-cv):
Podzbiór 1: Recall = 0.9667
Podzbiór 2: Recall = 0.9667
Podzbiór 3: Recall = 0.9333
Podzbiór 4: Recall = 0.9667
Podzbiór 5: Recall = 1.0000

Przeciętna czułość (5-cv): 0.9667

Przeciętny F1 (10-cv): 0.9659

Czas treningu (5-cv): 1.4206 sekundy

Zadanie 2
Fitting 5 folds for each of 175 candidates, totalling 875 fits
Przeciętny F1 score (5-cv): 0.9665

Wyniki F1 score dla każdego podzbioru (5-cv):
Podzbiór 1: F1 = 0.5232
Podzbiór 2: F1 = 0.7907
Podzbiór 3: F1 = 0.9662
Podzbiór 4: F1 = 0.9662
Podzbiór 5: F1 = 0.9598
Podzbiór 6: F1 = 0.9665
Podzbiór 7: F1 = 0.9665
Podzbiór 8: F1 = 0.9598
Podzbiór 9: F1 = 0.9393
Podzbiór 10: F1 = 0.9661
Podzbiór 11: F1 = 0.9665
Podzbiór 12: F1 = 0.9665
Podzbiór 13: F1 = 0.9665
Podzbiór 14: F1 = 0.9665
Podzbiór 15: F1 = 0.9665

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   7 out of   7 | elapsed:    1.4s finished
[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  10 out of  10 | elapsed:    0.0s finished
C:\Users\d_gar\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:318: UserWarning: The total space of parameters 16 is smaller than n_iter=100. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(



Wyniki testowe dla każdego podzbioru (R2):
Podzbiór 1: R2 = 0.3497
Podzbiór 2: R2 = 0.1989
Podzbiór 3: R2 = 0.5314
Podzbiór 4: R2 = 0.3479
Podzbiór 5: R2 = 0.3030
Podzbiór 6: R2 = 0.3399
Podzbiór 7: R2 = 0.4389

Przeciętny wynik R2: 0.3585

Wyniki testowe dla każdego podzbioru (MAE):
Podzbiór 1: MAE = 45.4906
Podzbiór 2: MAE = 50.4667
Podzbiór 3: MAE = 43.9778
Podzbiór 4: MAE = 49.7937
Podzbiór 5: MAE = 51.6603
Podzbiór 6: MAE = 46.1746
Podzbiór 7: MAE = 47.4921

Przeciętne MAE: 47.8651

Przeciętny wynik R2 (10-cv): 0.3329

Czas treningu (7-cv): 1.5089 sekundy

Zadanie 4
Fitting 5 folds for each of 16 candidates, totalling 80 fits

Najlepsza konfiguracja: {'n_estimators': 6, 'max_depth': 4}

Przeciętny wynik R2 (5-cv): 0.4102

Wyniki R2 dla każdego podzbioru:
Podzbiór 1: R2 = 0.2965
Podzbiór 2: R2 = 0.3280
Podzbiór 3: R2 = 0.3522
Podzbiór 4: R2 = 0.3926
Podzbiór 5: R2 = 0.3365
Podzbiór 6: R2 = 0.4102
Podzbiór 7: R2 = 0.2478
Podzbiór 8: R2 = 0.3246
Podzbiór 9: R2 = 0.2648
Podzbiór 10: 

C:\Users\d_gar\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:318: UserWarning: The total space of parameters 16 is smaller than n_iter=100. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(



Przeciętny wynik R2 (10-cv): 0.3714
Fitting 5 folds for each of 16 candidates, totalling 80 fits

Czas treningu (200 kombinacji, 5-cv): 0.1703 sekundy


C:\Users\d_gar\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py:318: UserWarning: The total space of parameters 16 is smaller than n_iter=200. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
